This notebook compares the performance of SignHunter and Harmonica on the two examples provided by Harmonica's authors.

To run this notebook, please copy the contents of this notebook's directory (`samplings.py`, `main.py`, and `SignHunter - Harmonica Comparison.ipynb`)  into the directory of the Harmonica's [source code](https://github.com/callowbird/Harmonica)

One should note that the two algorithms make different assumptions on the objective function to be optimized.
SignHunter assumes the function is separable in its variables, while Harmonica assumes that the function _can be approximated by a sparse and low degree polynomial in the Fourier basis. This
means intuitively that it can be approximated well by a decision tree_. See [this](https://arxiv.org/pdf/1706.00764.pdf)

From below, we can see that SignHunter minimizes the first objective function using 20 queries achieving a value of -50.0 whereas Harmonica does the same using 4223 queries!

For the second objective function, Signhunter achieves a better objective value of -916.2654 with 500 queries. On the other hand, Harmonica employs 4223 queries to obtain an objective value of -916.247764.

In terms of computational complexity per query, SignHunter is far more efficient. Please see the time per query from each cell below. E.g., SignHunter takes $36\mu s$ per query and Harmonica takes $67ms$!

Please note that we are using Harmonica's examples and its default parameters. There are no hyperparameter for SignHunter. 

In [1]:
import samplings
import numpy as np
import time

In [2]:
def sign_hunter(f, dim, max_num_queries):
    """
    A simple implementation of SignHunter
    """
    best_guess = np.ones(dim)
    best_f = f(best_guess)
    guess = best_guess.copy()
    num_queries = 1
    while num_queries < max_num_queries:
        for h in range(np.ceil(np.log2(dim)).astype(int) + 1):
            chunk_len = np.ceil(dim / 2**h).astype(int)
            for i in range(2**h):
                istart = i * chunk_len
                iend = min(istart + chunk_len, dim)
                guess[istart:iend] *= -1
                val = f(guess)
                num_queries += 1
                if val <= best_f:
                    #print("guess is better", istart, iend, best_guess, guess)
                    best_guess = guess.copy()
                    best_f = val
                else:
                    #print("guess is worse", istart, iend, best_guess, guess)
                    guess = best_guess.copy()
                #print(istart, iend)
                if istart == dim - 1:
                    break
                if num_queries >= max_num_queries:
                    break
            if num_queries >= max_num_queries:
                break
        if num_queries >= max_num_queries:
            break
        # signhunter assumes the objective function is separable so it returns a solution in O(n)
        # if the assumption is not satisfied we can start with a new guess after O(n) queries.
        guess = np.sign(np.random.randn(dim))
        num_queries += 1
        val = f(guess)
        if val <= best_f:
            best_guess = guess.copy()
            best_f = val
    return best_f, guess, num_queries

### Objective Function 1 from Harmonica source code

https://github.com/callowbird/Harmonica

###### SignHunter

In [3]:
samplings.num_queries = 0
query = samplings.f1
start_time = time.time()
best_f, sol, num_queries= sign_hunter(query, 60, 20)
end_time = time.time()
time_per_query = (end_time - start_time) / num_queries * 1e6
print("\n SignHunter: Number of queries {}| Solution cost {}| Time per query {:.2f}us".format(
    samplings.num_queries, best_f,time_per_query))


 SignHunter: Number of queries 20| Solution cost -50.0| Time per query 36.30us


###### Harmonica

In [4]:
!python main.py -query f1

Total options :  60
----------------------------------------------------------
Stage 0
Sampling..
Number of features :  36051
Extending feature vectors with degree 3 ..
Running linear regression..
Found the following sparse low degree polynomial:
 f = 7.35 x2 * x8 +7.14 x15 +6.90 x3 * x6 +6.73 x38 +6.68 x46
----------------------------------------------------------
Stage 1
Sampling..
Extending feature vectors with degree 3 ..
Running linear regression..
/Users/ash/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  positive)
Found the following sparse low degree polynomial:
 f = 0.00  +0.00 x17 * x27 * x28 +0.00 x17 * x26 * x59 +0.00 x17 * x26 * x58 +0.00 x17 * x26 * x57
----------------------------------------------------------
Stage 2
Sampling..
Extending feature vectors with degree 3 ..
Running linear regression..


### Objective Function 2 from Harmonica source code

https://github.com/callowbird/Harmonica

###### SignHunter

In [5]:
samplings.num_queries = 0
query = samplings.f2
start_time = time.time()
best_f, sol, num_queries= sign_hunter(query, 60, 500)
end_time = time.time()
time_per_query = (end_time - start_time) / num_queries * 1e6
print("\n SignHunter: Number of queries {}| Solution cost {}| Time per query {:.2f}us".format(
    samplings.num_queries, best_f,time_per_query))

C0	C2	C68	C0	C29	C937	C0	C11	C363	C0	C20	C655	C0	C24	C784	C0	C17	C561	C0	C31	C994	C0	C9	C289	C0	C2	C68	C0	C24	C793	C0	C3	C111	C0	C21	C697	C0	C18	C607	C0	C8	C267	C0	C11	C360	C0	C2	C84	C0	C2	C68	C0	C2	C68	C0	C4	C141	C0	C30	C988	C0	C3	C111	C0	C2	C70	C0	C16	C543	C0	C7	C248	C0	C2	C65	C0	C18	C607	C0	C2	C67	C0	C8	C280	C0	C3	C125	C0	C10	C324	C0	C2	C81	C0	C2	C65	C0	C2	C65	C0	C2	C65	C0	C2	C65	C0	C2	C65	C0	C4	C133	C0	C2	C65	C0	C10	C332	C0	C22	C713	C0	C3	C122	C0	C2	C65	C0	C2	C67	C0	C2	C65	C0	C16	C518	C0	C2	C73	C0	C2	C69	C0	C7	C242	C0	C2	C69	C0	C2	C64	C0	C2	C65	C0	C18	C601	C0	C2	C66	C0	C2	C64	C0	C10	C333	C0	C0	C12	C0	C1	C55	C0	C0	C28	C0	C8	C272	C0	C0	C12	C0	C0	C12	C0	C0	C12	C0	C0	C12	C0	C0	C12	C0	C0	C4	C0	C0	C4	C0	C0	C4	C0	C0	C4	C0	C0	C4	C0	C0	C4	C0	C0	C4	C0	C0	C12	C0	C2	C80	C0	C4	C146	C0	C0	C4	C0	C0	C0	C0	C0	C0	C0	C8	C262	C0	C20	C651	C0	C0	C0	C0	C0	C0	C0	C1	C52	C0	C0	C2	C0	C0	C0	C0	C0	C0	C0	C0	C18	C0	C0	C0	C0	C0	C0	C0	C16	C538	C0	C2	C82	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C5	C169	C0	C0	C0	C0	C0

###### Harmonica

In [6]:
!python main.py -query f2

Total options :  60
----------------------------------------------------------
Stage 0
Sampling..
C0	C6	C192	C0	C8	C266	C0	C20	C641	C0	C24	C781	C0	C5	C179	C0	C31	C1019	C0	C31	C996	C0	C24	C785	C0	C18	C596	C0	C8	C286	C0	C16	C531	C0	C8	C268	C0	C9	C291	C0	C19	C626	C0	C7	C236	C0	C19	C634	C0	C26	C847	C0	C12	C412	C0	C0	C5	C0	C19	C611	C0	C3	C101	C0	C29	C938	C0	C31	C1012	C0	C21	C683	C0	C0	C2	C0	C23	C761	C0	C9	C299	C0	C13	C439	C0	C20	C667	C0	C25	C831	C0	C12	C411	C0	C24	C772	C0	C19	C629	C0	C19	C632	C0	C22	C719	C0	C8	C262	C0	C19	C621	C0	C28	C896	C0	C12	C385	C0	C29	C941	C0	C12	C405	C0	C13	C431	C0	C5	C165	C0	C30	C962	C0	C21	C700	C0	C16	C535	C0	C16	C514	C0	C14	C456	C0	C1	C37	C0	C8	C270	C0	C21	C674	C0	C1	C49	C0	C9	C306	C0	C22	C720	C0	C26	C859	C0	C31	C1000	C0	C12	C403	C0	C14	C467	C0	C29	C958	C0	C8	C282	C0	C21	C694	C0	C17	C550	C0	C16	C518	C0	C13	C447	C0	C11	C352	C0	C21	C688	C0	C17	C561	C0	C1	C46	C0	C28	C918	C0	C28	C900	C0	C14	C478	C0	C9	C305	C0	C3	C122	C0	C31	C1014	C0	C5	C162	C0	C25	C807	C0	C6	C208	C0	C

Running linear regression..
Found the following sparse low degree polynomial:
 f = -5.39 x22 +5.01 x20 * x45-4.87 x6 * x27-2.81 x15 +2.60 x16 * x50 * x59
----------------------------------------------------------
----------------------------------------------------------
Running hyperband as the base algorithm.
B= 8000
Smax= 4
WARNING! Please implement the intermediate sampling algorithm.
The current intermediate sampling algorithm is trivial. It cannot be applied to your application.
----------------------------------------------------------
s= 0
n= 80
Remaining.. 80
r= 100
C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C

Remaining.. 1296
r= 1
C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	

C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C0	C